In [1]:
from pandas_datareader import data as web

import plotly.express as px

import pandas as pd
import numpy as np

# Tarefa 02

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

#### 1. Carregue os dados da tarefa anterior

In [2]:
ativos = ['U', 'NVDA', 'TSLA']
dt_ini = '01/01/2012'
yf = web.get_data_yahoo(ativos, start=dt_ini)

In [3]:
amp_columns = pd.MultiIndex.from_tuples(tuples=[
    ('Amplitude', 'U'),
    ('Amplitude', 'NVDA'),
    ('Amplitude', 'TSLA')
])
amplitude = (yf['High'] - yf['Low'])
amplitude.columns = amp_columns
yf = yf.join(amplitude)

In [4]:
yf.head()

Attributes Adj Close                     Close                   High          \
Symbols            U      NVDA      TSLA     U    NVDA      TSLA    U    NVDA   
Date                                                                            
2012-01-03       NaN  3.222135  1.872000   NaN  3.5100  1.872000  NaN  3.6000   
2012-01-04       NaN  3.258853  1.847333   NaN  3.5500  1.847333  NaN  3.5650   
2012-01-05       NaN  3.375898  1.808000   NaN  3.6775  1.808000  NaN  3.6950   
2012-01-06       NaN  3.336883  1.794000   NaN  3.6350  1.794000  NaN  3.6775   
2012-01-09       NaN  3.336883  1.816667   NaN  3.6350  1.816667  NaN  3.7050   

Attributes           Low  ...           Open                   Volume  \
Symbols         TSLA   U  ...      TSLA    U    NVDA      TSLA      U   
Date                      ...                                           
2012-01-03  1.966667 NaN  ...  1.843333  NaN  3.5750  1.929333    NaN   
2012-01-04  1.911333 NaN  ...  1.833333  NaN  3.5125  1.880667    NaN   
2012-01-05  1.862000 NaN  ...  1.790000  NaN  3.5325  1.850667    NaN   
2012-01-06  1.852667 NaN  ...  1.760667  NaN  3.6750  1.813333    NaN   
2012-01-09  1.832667 NaN  ...  1.741333  NaN  3.6375  1.800000    NaN   

Attributes                         Amplitude                    
Symbols           NVDA        TSLA         U    NVDA      TSLA  
Date                                                            
2012-01-03  46804400.0  13921500.0       NaN  0.0975  0.123334  
2012-01-04  34737200.0   9451500.0       NaN  0.0850  0.078000  
2012-01-05  56354800.0  15082500.0       NaN  0.1775  0.072000  
2012-01-06  53325200.0  14794500.0       NaN  0.0850  0.092000  
2012-01-09  50824400.0  13455000.0       NaN  0.0950  0.091334  

[5 rows x 21 columns]

#### 2. Média móvel

1. Faça uma média móvel  em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu
2. Utilizando a fução ```rolling()```, calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
2. Monte um gráfico de linhas para as médias móveis dos ativos da sua base
4. Monte um gráfico de linhas para os desvios padrão em janela móvel
5. Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

In [5]:
window = yf['Close'].rolling(90)

In [6]:
rolling_avg = window.mean()
rolling_std = window.std()

In [7]:
rolling_avg_vert = rolling_avg.stack().reset_index()
rolling_avg_vert.columns = ['Data', 'Ativo', 'Média']
rolling_std_vert = rolling_std.stack().reset_index()
rolling_std_vert.columns = ['Data', 'Ativo', 'Desvio']
#rolling_std_vert = rolling_avg.stack().reset_index()

In [8]:
fig = px.line(
    data_frame=rolling_avg_vert,
    x='Data',
    y='Média',
    color='Ativo',
    template='plotly_white'
)
fig.show()

In [9]:
fig = px.line(
    data_frame=rolling_std_vert,
    x='Data',
    y='Desvio',
    color='Ativo',
    template='plotly_white'
)
fig.show()

A Tesla possui tanto o maior valor da ação como a maior volatilidade absoluta nos últimos meses.

#### 3. Gráfico de *candlestick*
1. Selecione o período dos últimos 60 dias (corridos) para um dos ativos da sua base
2. Monte um gráfico de *candlestick* para esses dados.

In [10]:
indicadores = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
ativo = ['NVDA'] * len(indicadores)
colunas = list(zip(indicadores, ativo))
nvda = yf.loc[yf.index[-60]:,colunas]
nvda.columns = indicadores

In [11]:
from plotly import graph_objects
from plotly.graph_objs import Layout

In [19]:
graph_data = {
    'x': nvda.index,
    'open': nvda.Open,
    'close': nvda.Close,
    'high': nvda.High,
    'low': nvda.Low,
    'type': 'candlestick',
    'name': 'NVDA',
    'showlegend': True
}
graph_layout = Layout(
    paper_bgcolor='rgba(255,255,255,255)',
    plot_bgcolor='rgba(255,255,255,255)'
)

In [20]:
graph = graph_objects.Figure(
    data=[graph_data],
    layout_title='Nvidia',
    layout=graph_layout
)
graph.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
graph.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
graph